## ComCam Astrometry

Finding an a.net solution for one of the first images
Craig Lage - 25-Oct-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.summit.utils.plotting import plot
import lsst.afw.cameraGeom.utils as camGeomUtils

from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u
from astropy.table import Table
from astroquery.astrometry_net import AstrometryNet
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig

In [ ]:
butler = Butler('/repo/embargo_new', collections=["LSSTComCam/raw/all", "LSSTComCam/calib", "LSSTComCam/nightlyValidation"])

## Get the postISRCCD data

In [ ]:
expId = 2024102600106
instrument = 'LSSTComCam'
exp = butler.get('postISRCCD', detector=4, exposure=expId, instrument=instrument)
%matplotlib inline
x = plot(exp, stretch='ccs')
x.axes[0].set_title(f"ComCam {expId} Det 4", fontsize=18)
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_postISRCD_{expId}.png")

# Find the sources

In [ ]:
charConfig = CharacterizeImageConfig()
charConfig.doMeasurePsf = True
charConfig.doApCorr = False
charConfig.doDeblend = False
charConfig.repair.doCosmicRay = False
charConfig.repair.doInterpolate = True   
charConfig.detection.minPixels = 500
charTask = CharacterizeImageTask(config=charConfig)

mData = exp.getMetadata()
charResult = charTask.run(exp)
sourceCatalog = charResult.sourceCat


In [ ]:
sources = sourceCatalog.asAstropy()
sources.keep_columns(['base_SdssCentroid_x', 'base_SdssCentroid_y', 'base_CircularApertureFlux_3_0_instFlux'])
sources.sort('base_CircularApertureFlux_3_0_instFlux', reverse=True)
len(sources)

In [ ]:
ast = AstrometryNet()
ast.api_key = 'xxawwhvleirxcswx'

In [ ]:
image_width = 4072
image_height = 4000
scale_units = 'arcsecperpix'
scale_type='ev' # ev means submit estimate and % error
scale_est = 0.20
scale_err = 2.0
center_ra = mData['RA']
center_dec = mData['DEC']
radius = 0.5
wcs_header = ast.solve_from_source_list(sources['base_SdssCentroid_x'], sources['base_SdssCentroid_y'],
                                        image_width, image_height, scale_units=scale_units,
                                        scale_type=scale_type, scale_est=scale_est, scale_err=scale_err,
                                        center_ra=center_ra, center_dec=center_dec, radius=radius,
                                        crpix_center=True, solve_timeout=240)

In [ ]:
wcs_header

In [ ]:
print(f"Header center RA = {mData['RA']:.5f}, Anet center RA = {wcs_header['CRVAL1']:.5f}")
print(f"Header center Dec = {mData['DEC']:.5f}, Anet center Dec = {wcs_header['CRVAL2']:.5f}")

In [ ]:
RA_error = (wcs_header['CRVAL1'] - mData['RA']) * 3600.0
Dec_error = (wcs_header['CRVAL2'] - mData['DEC']) * 3600.0
print(f"RA error = {RA_error:.1f} arcsec, Dec error = {Dec_error:.1f} arcsec")

In [ ]:
140/3600

In [ ]:
x = plot(exp, stretch='ccs')
text1 = f"Header center RA = {mData['RA']:.5f}, Anet center RA = {wcs_header['CRVAL1']:.5f}"
text2 = f"Header center Dec = {mData['DEC']:.5f}, Anet center Dec = {wcs_header['CRVAL2']:.5f}"
RA_error = (wcs_header['CRVAL1'] - mData['RA']) * 3600.0
Dec_error = (wcs_header['CRVAL2'] - mData['DEC']) * 3600.0
text3 = f"RA error = {RA_error:.1f} arcsec, Dec error = {Dec_error:.1f} arcsec"
x.axes[0].text(800,1800, text1, color="cyan", fontsize=12, weight='bold')
x.axes[0].text(800,1600, text2, color="cyan", fontsize=12, weight='bold')
x.axes[0].text(800,1400, text3, color="cyan", fontsize=12, weight='bold')
x.axes[0].set_title(f"ComCam Astrometry {expId}", fontsize=24)
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Astrometry{expId}.png")

In [ ]:
for key in mData.keys():
    print(key, mData[key])

In [ ]:
exp.getWcs()

In [ ]:
expId = 2024102600179
instrument = 'LSSTComCam'
exp = butler.get('postISRCCD', detector=4, exposure=expId, instrument=instrument)
%matplotlib inline
x1=plot(exp, stretch='ccs')
x1.axes[0].set_title(f"ComCam {expId} Det 4", fontsize=18)
expId = 2024102600197
instrument = 'LSSTComCam'
exp = butler.get('postISRCCD', detector=4, exposure=expId, instrument=instrument)
%matplotlib inline
x2=plot(exp, stretch='ccs')
x2.axes[0].set_title(f"ComCam {expId} Det 4", fontsize=18)
#plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_postISRCD_{expId}.png")

# Find the sources

In [ ]:
expId = 2024102600183
instrument = 'LSSTComCam'
exp = butler.get('postISRCCD', detector=4, exposure=expId, instrument=instrument)

charConfig = CharacterizeImageConfig()
charConfig.doMeasurePsf = False
charConfig.doApCorr = False
charConfig.doDeblend = False
charConfig.repair.doCosmicRay = False
charConfig.repair.doInterpolate = False   
charConfig.detection.minPixels = 10000
charConfig.doNormalizedCalibration=False
charTask = CharacterizeImageTask(config=charConfig)

mData = exp.getMetadata()
charResult = charTask.run(exp)
sourceCatalog = charResult.sourceCat


In [ ]:
sources = sourceCatalog.asAstropy()
sources.keep_columns(['base_SdssCentroid_x', 'base_SdssCentroid_y', 'base_CircularApertureFlux_3_0_instFlux'])
sources.sort('base_CircularApertureFlux_3_0_instFlux', reverse=True)
len(sources)

In [ ]:
def plotStamp(sources, index):
    stamp1 = 250
    x11 = int(sources[index]['base_SdssCentroid_x']) - stamp1
    x21 = int(sources[index]['base_SdssCentroid_x']) + stamp1
    y11 = int(sources[index]['base_SdssCentroid_y']) - stamp1
    y21 = int(sources[index]['base_SdssCentroid_y']) + stamp1
    
    first = exp.image.array[y11:y21,x11:x21]
    first -= np.median(first)    

    sumx = 0.0
    sumy = 0.0
    for i in range(first.shape[0]):
        for j in range(first.shape[1]):
            sumx += i * first[i,j]
            sumy += j * first[i,j]
    sumx /= np.sum(first)
    sumy /= np.sum(first)
    stamp2 = 120
    x12 = int(sumx-stamp2)
    x22 = int(sumx+stamp2)
    y12 = int(sumy-stamp2)
    y22 = int(sumy+stamp2)
    
    second = first[x12:x22, y12:y22]
    plt.imshow(test2)
    #plt.colorbar()
    return


In [ ]:
plotStamp(sources, 0)